In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf

ModelCheckpoint = tf.keras.callbacks.ModelCheckpoint
load_model = tf.keras.models.load_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sqlite3

import plotly.express as px
import plotly
import plotly.graph_objs as go

import warnings
warnings.filterwarnings('ignore')

In [ ]:
rating = pd.read_csv('BX-Book-Ratings.csv', sep=';', encoding="latin-1")

print(rating.shape)
rating.head()

In [ ]:
user = pd.read_csv('BX-Users.csv', sep=';', encoding="latin-1")

print(user.shape)
user.head()

In [ ]:
book = pd.read_csv('BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")

print(book.shape)
book.head()

In [ ]:
connect = sqlite3.connect('book_rec.db')

In [ ]:
book.to_sql("book", connect, if_exists='fail')
user.to_sql("user", connect, if_exists='fail')
rating.to_sql("rating", connect, if_exists='fail')

In [ ]:
pip install ipython-sql

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///book_rec.db

In [ ]:
%sql SELECT * FROM rating LIMIT 5

In [ ]:
%sql SELECT * FROM book LIMIT 5

In [ ]:
%sql SELECT * FROM user LIMIT 5

In [ ]:
%%sql
CREATE TABLE rating_book AS SELECT rating."User-ID" AS UserID , rating.ISBN, rating."Book-Rating" AS BookRating, book."Book-Title" AS BookTitle
FROM rating 
INNER JOIN book
ON rating.ISBN = book.ISBN

In [ ]:
%%sql
SELECT * FROM rating_book LIMIT 5

In [ ]:
%%sql
SELECT count(*) FROM rating_book

In [ ]:
%%sql
CREATE TABLE rating_count AS SELECT "BookTitle", count(*) as RatingCountBook FROM rating_book GROUP BY "BookTitle"

In [ ]:
%%sql
SELECT * from rating_count limit 5

In [ ]:
%%sql
SELECT count(*) FROM rating_count

In [ ]:
rating_coun_threshold = 20

In [ ]:
%%sql
CREATE TABLE rating_count_filter AS SELECT * FROM rating_count WHERE RatingCountBook >= :rating_coun_threshold

In [ ]:
%%sql
SELECT * FROM rating_count_filter LIMIT 5

In [ ]:
%%sql
SELECT count(*) FROM rating_count_filter

In [ ]:
%%sql
CREATE TABLE user_rating AS SELECT rating_book.UserID, rating_book.ISBN, rating_book.BookRating, rating_book.BookTitle, rating_count_filter.RatingCountBook
FROM rating_count_filter 
LEFT JOIN rating_book
ON rating_count_filter.BookTitle = rating_book.BookTitle

In [ ]:
%%sql
SELECT * FROM user_rating LIMIT 5

In [ ]:
%%sql
SELECT count(*) FROM user_rating

In [ ]:
%%sql
CREATE TABLE user_count AS SELECT UserID, count(*) as RatingCountUser FROM user_rating GROUP BY UserID

In [ ]:
%%sql
SELECT * FROM user_count LIMIT 5

In [ ]:
%%sql
SELECT count(*) FROM user_count

In [ ]:
user_count_filter_threshold = 20

In [ ]:
%%sql
CREATE TABLE user_count_filter AS SELECT * FROM user_count WHERE RatingCountUser >= :user_count_filter_threshold

In [ ]:
%%sql
SELECT * FROM user_count_filter LIMIT 5

In [ ]:
%%sql
SELECT count(*) FROM user_count_filter

In [ ]:
%%sql
CREATE TABLE combined AS SELECT user_rating.BookTitle, user_rating.RatingCountBook, user_rating.UserID, user_rating.ISBN, user_rating.BookRating, user_count_filter.RatingCountUser
FROM user_rating
INNER JOIN user_count_filter
ON user_rating.UserID = user_count_filter.UserID

In [ ]:
%%sql
SELECT * FROM combined LIMIT 5

In [ ]:
%%sql
SELECT count(*) FROM combined

In [ ]:
%%sql
select count(distinct BookTitle) AS NumberOfUniqueBooks
FROM combined

In [ ]:
%%sql
select count(distinct UserID) AS NumberOfUniqueUsers
FROM combined

# **Transform to Dataset**

In [ ]:
combined = %sql SELECT * FROM combined

In [ ]:
combined_df = pd.DataFrame(data = combined, columns = combined.field_names)
combined_df.head()

In [ ]:
combined_df['BookRating'] = combined_df['BookRating'].values.astype(float)

print(combined_df.shape)
combined_df.head()

In [ ]:
combined_df = combined_df.drop_duplicates(['UserID', 'BookTitle'])
print(combined_df.shape)

In [ ]:
user_book_matrix = combined_df.pivot(index='UserID', columns='BookTitle', values='BookRating')

print(user_book_matrix.shape)
user_book_matrix.head()

In [ ]:
user_book_matrix.fillna(0, inplace=True)
user_book_matrix.head()

In [ ]:
users = user_book_matrix.index.tolist()
users[0:10]

In [ ]:
books = user_book_matrix.columns.tolist()
books[0:10]

In [ ]:
user_book_matrix = user_book_matrix.to_numpy()
user_book_matrix.shape

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(user_book_matrix, user_book_matrix, test_size=0.2, random_state=1)

In [ ]:
t = MinMaxScaler()

t.fit(user_book_matrix)
x_train = t.transform(x_train)
x_val = t.transform(x_val)

In [ ]:
print(x_train.min(), x_train.max())
print(x_val.min(), x_val.max())

# **Autoencoder Model**

In [ ]:
num_input = len(books)

inp = tf.keras.layers.Input((num_input))
e = tf.keras.layers.Dense(10)(inp)
e = tf.keras.layers.BatchNormalization()(e)
e = tf.keras.layers.LeakyReLU()(e)

n_bottleneck = 5
bottleneck = tf.keras.layers.Dense(n_bottleneck)(e)

In [ ]:
d = tf.keras.layers.Dense(10)(bottleneck)
d = tf.keras.layers.BatchNormalization()(d)
d = tf.keras.layers.LeakyReLU()(d)

decoded = tf.keras.layers.Dense(num_input, activation='sigmoid')(d)
                
ae = tf.keras.models.Model(inp, decoded)
ae.summary()

In [ ]:
ae.compile(loss='mse', optimizer='adam')

In [ ]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
history = ae.fit(x_train, x_train, epochs=300, verbose=1, batch_size=32, validation_data=(x_val, x_val), callbacks = [checkpoint], shuffle= True)

In [ ]:
h1 = go.Scatter(y=history.history['loss'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='blue'),
                        name="loss"
                   )
h2 = go.Scatter(y=history.history['val_loss'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='red'),
                        name="val_loss"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1)


In [ ]:
predict_model = load_model(filename) 
predict_model.summary()

In [ ]:
print(user_book_matrix.shape)
preds = predict_model(user_book_matrix)

In [ ]:
preds = preds.numpy()

pred_data = pd.DataFrame(preds)
pred_data.head()

In [ ]:
pred_data = pred_data.stack().reset_index()
pred_data.head()

In [ ]:
pred_data.columns = ['UserID', 'BookTitle', 'BookRating']

print(pred_data.shape)
pred_data.head()

In [ ]:
pred_data['UserID'] = pred_data['UserID'].map(lambda value: users[value])

pred_data.head()

In [ ]:
pred_data['BookTitle'] = pred_data['BookTitle'].map(lambda value: books[value])

pred_data.head()

In [ ]:
pred_data.to_sql("pred_data", connect, if_exists='fail')

# **Query Top 10 Book Rating**

In [ ]:
%%sql
SELECT * 
FROM pred_data 
WHERE UserID=243
ORDER BY BookRating DESC
LIMIT 10

In [ ]:
%%sql
SELECT * 
FROM rating_book
WHERE UserID=243
ORDER BY BookRating DESC
LIMIT 10